<a href="https://colab.research.google.com/github/marcelo-rossi/IARPG_Master/blob/main/IARPG_Master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

### Install

In [1]:
!pip install -U -q google-generativeai

### Imports

In [2]:
# import necessary modules.
import google.generativeai as genai
import json
import base64
import pathlib
import pprint
import requests
import mimetypes
from IPython.display import Markdown
import time
import logging

logging.getLogger('tornado.access').setLevel(logging.ERROR) # Define o nível de log para ERROR ou CRITICAL

### Configurando a chave da API (API key)

In [3]:
from google.colab import userdata
API_KEY=userdata.get('api_key')
genai.configure(api_key=API_KEY)

## Configurando o Modelo

In [4]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [5]:
generation_config = {
    'candidate_count': 1,
    'temperature': 1,
}

In [6]:
safety_settings = {
    'HARASSMENT': 0,
    'HATE': 0,
    'SEXUAL': 0,
    'DANGEROUS': 0,
}

In [7]:
model = genai.GenerativeModel(
    model_name='gemini-1.5-pro-latest',
    generation_config=generation_config,
    safety_settings=safety_settings
)
delay = 1

## Bora pro o Jogo!

##Está a fim de se divertir.
Que tal participar de uma aventura de RPG? Este projeto te leva a uma aventura em um ambiente medieval onde você como protagonista deseja entrar para uma guilda de ladrões, mas para isso você deverá passar pelo seu primeiro desafio antes de ser aceito. Topa jogar este jogo único criado para você e só para este momento? A cada retorno você irá curtir um jogo completamente novo. Rode o código e divirta-se

In [8]:
chat = model.start_chat(history=[])

In [ ]:
# Definição da variável delay
start_delay = 1

def handle_inappropriate_content(content, prompt):
  print('Conteúdo inadequado gerado, estamos adequando ele para um jogo melhor')
  content = 'O ultimo conteúdo que você gerou foi inapropriado. Gere um novo conteúdo baseado na história até o momento e ' + content + prompt
  response = chat.send_message(content)
  candidate = response.candidates[0]
  time.sleep(delay)
  return response, candidate

def handle_send_message(content):
  ok = False
  delay = start_delay
  while not ok:
    try:
      response = chat.send_message(content)
      candidate = response.candidates[0]
      time.sleep(delay)
      ok = True
      # delay = start_delay
    except:
      # error_message = f'Estamos tentando criar um bom roteiro, aguarde um momento por favor.'
      # display(Markdown(error_message))
      delay *= 2
      if delay > 20:
        delay = 20
  return response, candidate

# Início do jogo
content = '''Você é um mestre de RPG de mesa. Desenvolva um jogo para só uma pessoa jogar com você. A partir de agora você é o chefe de uma guilda de ladrões. A sua forma de falar é a de um carioca marrento. O seu sub-chefe tem a forma de falar de um mineiro contador de proza. Invente um nome para você e para o seu sub-chefe. A partir de agora quando eu me referir a você ou eu vou te chamar pelo nome que você inventou ou vou te chamar por [Chefe]. O mesmo valor para o seu sub-chefe, no qual eu irei chamar de [Sub-Chefe] ou pelo nome que você criou. A história inicia com [Chefe] e o [Sub-Chefe] se apresentando, apresentado a guilda e falando um pouco das operações que a guia faz. O jogador está querendo entrar para a guilda. Você deve indicar que no bosque nos arredores da vila tem uma mansão e para que o jogador possa entrar na guilda ele deve roubar uma determinada joia que se encontra na mansão. Invente uma autoridade que seja o dono da mansão e de um nome para a mansão e para a autoridade. Invente qual é a joia que o jogador deve roubar, pode ser um colar, ou um anel, ou um bracelete, ou qualquer outro adorno e de um nome para esta joia. Invente um motivo para que os proprietários da mansão não estejam presentes e que a vigilância da mansão está baixa e é por isso que o momento de roubar a joia deve ser agora. Redija o texto na forma de diálogo e separe o diálogo entre parágrafos, você também deve indicar qual personagem está falando e faça isso colocando o nome dele entre [] no início do parágrafo. O [Chefe] e o [Sub-Chefe] não irão acompanhar o jogador na sua jornada se precisar narrar a cena indique [Narrador]. Ao longo do jogo você deve dar no máximo 4 opções de ações ao jogador e deve apresentar essas opções em tópicos enumerados e o jogador só vai poder escolher um número e você cria o que aconteceu na história baseado na escolha dele. A única exceção são as seções de diálogos onde poderão ter até 5 opções, sendo uma delas obrigatoriamente a opção de dialogar com os personagens. Durante os diálogos o jogador poderá fazer perguntas aos personagens e eles responderão com diálogos curtos, basicamente de no máximo 4 trocas de falas no qual os personagens encerrarão o diálogo indicando que o jogador deve agir e você, sendo o mestre do jogo, volta a apresentar as opções de ações que o jogador tem. A história se passa em um ambiente medieval, sem magia, sem tecnologia. Nenhum personagem tem conhecimento de magia ou de tecnologias que não fazem parte de um ambiente medieval. Qualquer texto que eu envie que não condiz com a história deve ser indicado que não é adequado ao contexto. Se o personagem do jogador for preso ou morto é fim de jogo. Se o jogador desistir é fim de jogo. Nunca pergunte o nome do jogador.'''
response, candidate = handle_send_message(content)
time.sleep(delay)

# Tratar conteúdo inadequado no início
while len(candidate.safety_ratings) == 0:
  response, candidate = handle_inappropriate_content(content, '')

display(Markdown(response.text))

game_over = False
offencive_gameplay = False
while not game_over or not offencive_gameplay:
  display(Markdown("**Escolha uma das opções ou dê a sua sugestão de ação.**"))
  prompt = input('O que você irá fazer? ')

  # Verificar fim de jogo
  content = 'o jogador escolheu "' + prompt + '" a escolha dele levou ao fim do jogo? Responda apenas sim ou não. Não gere mais conteúdo apenas responda sim ou não.'
  response, candidate = handle_send_message(content)
  time.sleep(delay)
  if response.text.lower() == 'sim \n':
    #print('Fim de jogo!')
    game_over = True

  # Verificar conteúdo do jogador
  content = 'o jogador escolheu "' + prompt + '". Analise se a mensagem dele contém linguagem inapropriada? Responda apenas sim ou não. Não gere mais conteúdo apenas responda sim ou não.'
  response, candidate = handle_send_message(content)
  time.sleep(delay)
  if response.text.lower() == 'sim \n':
    #print('Fim de jogo!')
    offencive_gameplay = True

  # Lidar com conteúdo inadequado na verificação de fim de jogo
  while len(candidate.safety_ratings) == 0:
    response, candidate = handle_inappropriate_content('', prompt)

  if game_over:

    # Gerar final do jogo
    content = '''Quando o jogador escolheu "''' + prompt + '''" você indicou que o jogo acabou. Crie um final para descrever o que aconteceu.'''
    response, candidate = handle_send_message(content)
    time.sleep(delay)

    # Lidar com conteúdo inadequado no final do jogo
    while len(candidate.safety_ratings) == 0:
      response, candidate = handle_inappropriate_content(content, '')

    display(Markdown(response.text))
    break

  if offencive_gameplay:

    # Gerar Finalizando o jogo por ações inadequadas indicadas pelo jogador
    content = '''O jogo acabou. Indique que o usuário desejou realizar foi algo inadquado e dê exemplo de como ele pode jogar de forma mais amistosa.'''
    response, candidate = handle_send_message(content)
    time.sleep(delay)

    # Lidar com conteúdo inadequado no final do jogo
    while len(candidate.safety_ratings) == 0:
      response, candidate = handle_inappropriate_content(content, '')

    display(Markdown(response.text))
    break


  content = '''o jogador escolheu "''' + prompt + '''" gere mais conteúdo para o jogo baseado na resposta dele. Independente do que o jogador escreveu forneça no máximo 4 opções de ações ao jogador e deve apresentar essas opções em tópicos enumerados e você cria o que aconteceu na história baseado na escolha dele.'''
  response, candidate = handle_send_message(content)
  time.sleep(delay)

  # Lidar com conteúdo inadequado no novo conteúdo
  while len(candidate.safety_ratings) == 0:
    response, candidate = handle_inappropriate_content(content, '')

  display(Markdown(response.text))